# Extract, Transform and Load

In [1]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
import csv
import matplotlib.pyplot as plt
import multiprocessing
from itertools import chain, islice
from datetime import timedelta
import jsonlines
import seaborn as sns
from pathlib import Path

### Mount Google Drive and Load Datasets

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
# Set the directory where the data is loaded.
local_dir = '/content/gdrive/MyDrive/Meli Data Challenge/'

In [4]:
df = pd.read_parquet(f'{local_dir}/train_data.parquet')

In [5]:
df.head()

,sku,date,sold_quantity,current_price,currency,listing_type,shipping_logistic_type,shipping_payment,minutes_active
0,464801,2021-02-01,0,156.78,REA,classic,fulfillment,free_shipping,1440.0
1,464801,2021-02-02,0,156.78,REA,classic,fulfillment,free_shipping,1440.0
2,464801,2021-02-03,0,156.78,REA,classic,fulfillment,free_shipping,1440.0
3,464801,2021-02-04,0,156.78,REA,classic,fulfillment,free_shipping,1440.0
4,464801,2021-02-05,1,156.78,REA,classic,fulfillment,free_shipping,1440.0


## Load extra item data

In [7]:
def load_jsonlines(filename):

  rv = []
  for obj in tqdm(jsonlines.open(filename)):
    rv.append(obj)
  return rv

In [8]:
item_metadata = load_jsonlines(f'{local_dir}items_static_metadata_full.jl')

660916it [00:05, 123176.30it/s]


### Convert to a df and use sku as the index

In [9]:
df_metadata = pd.DataFrame(item_metadata)
df_metadata.index = df_metadata.sku
df_metadata.drop(columns=['sku'], inplace=True)

In [10]:
df_metadata.head()

,item_domain_id,item_id,item_title,site_id,product_id,product_family_id
sku,,,,,,
0,MLB-SNEAKERS,492155,Tênis Masculino Olympikus Cyber Barato Promoçao,MLB,None,MLB15832732
1,MLB-SURFBOARD_RACKS,300279,Suporte Rack Prancha Parede C/ Regulagem Horiz...,MLB,None,None
2,MLM-NECKLACES,69847,5 Collares Plateados Dama Gargantilla Choker -...,MLM,None,None
3,MLM-RINGS,298603,Lindo Anillo De Bella Crepusculo Twilight Prom...,MLM,None,None
4,MLB-WEBCAMS,345949,Webcam Com Microfone Hd 720p Knup Youtube Pc V...,MLB,None,None


### Sample the data

In [11]:
df_sample = df.sample(frac=0.10, random_state=1)

#### Hydrate the initial datasets with the extra data

In [12]:
df_sample = df_sample.join(df_metadata, on='sku', how='left')

### Save as csv

In [13]:
df_sample.to_csv(f'{local_dir}df_sample.csv', index=False)

### Check that data has been saved correctly

In [14]:
ds = pd.read_csv(f'{local_dir}df_sample.csv')
ds.head()

,sku,date,sold_quantity,current_price,currency,listing_type,shipping_logistic_type,shipping_payment,minutes_active,item_domain_id,item_id,item_title,site_id,product_id,product_family_id
0,152848,2021-03-24,1,167.00,REA,premium,fulfillment,free_shipping,1041.411833,MLB-DRESSES,3892,Vestido Midi Mulher Virtuosa,MLB,NaN,NaN
1,431324,2021-02-09,27,28.49,REA,premium,fulfillment,paid_shipping,1440.000000,MLB-CELLPHONE_PARTS,56302,Cola Preta T7000 15ml Para Celular Reparo Touc...,MLB,NaN,NaN
2,525309,2021-03-04,0,37.91,REA,premium,fulfillment,paid_shipping,1440.000000,MLB-VEHICLE_STICKERS,494687,Kit Adesivos Dodge Dakota Sport 3.9 V6 Em Prat...,MLB,NaN,NaN
3,492760,2021-02-18,0,169.15,MEX,premium,drop_off,paid_shipping,1440.000000,MLM-CELLPHONE_COVERS,190467,Fundas Nintendo Switch Para iPhone 11 iPhone 1...,MLM,NaN,NaN
4,223511,2021-02-17,0,24.90,REA,premium,fulfillment,paid_shipping,0.000000,MLB-KITCHEN_SUPPLIES,467072,Cesto Cozimento A Vapor Inox Legumes Cozinha P...,MLB,NaN,NaN
